<a href="https://colab.research.google.com/github/santosh50/Text_to_Emoji/blob/main/sentence_to_emoji.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import numpy as np
import emoji
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()


def read_csv(filename = 'data/emojify_data.csv'):
    phrase = []
    emoji = []

    with open (filename) as csvDataFile:
        csvReader = csv.reader(csvDataFile)

        for row in csvReader:
            phrase.append(row[0])
            emoji.append(row[1])

    X = np.asarray(phrase)
    Y = np.asarray(emoji, dtype=int)

    return X, Y

def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y


emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":smile:",
                    "3": ":disappointed:",
                    "4": ":fork_and_knife:"}

def label_to_emoji(label):
    """
    Converts a label (int or string) into the corresponding emoji code (string) ready to be printed
    """
    return emoji.emojize(emoji_dictionary[str(label)], use_aliases=True
)              
    
def print_predictions(X, pred):
    print()
    for i in range(X.shape[0]):
        print(X[i], label_to_emoji(int(pred[i])))
        
        
def plot_confusion_matrix(y_actu, y_pred, title='Confusion matrix', cmap=plt.cm.gray_r):
    
    df_confusion = pd.crosstab(y_actu, y_pred.reshape(y_pred.shape[0],), rownames=['Actual'], colnames=['Predicted'], margins=True)
    
    df_conf_norm = df_confusion / df_confusion.sum(axis=1)
    
    plt.matshow(df_confusion, cmap=cmap) # imshow
    #plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(df_confusion.columns))
    plt.xticks(tick_marks, df_confusion.columns, rotation=45)
    plt.yticks(tick_marks, df_confusion.index)
    #plt.tight_layout()
    plt.ylabel(df_confusion.index.name)
    plt.xlabel(df_confusion.columns.name)
    
    
def predict(X, Y, W, b, word_to_vec_map):
    """
    Given X (sentences) and Y (emoji indices), predict emojis and compute the accuracy of your model over the given set.
    
    Arguments:
    X -- input data containing sentences, numpy array of shape (m, None)
    Y -- labels, containing index of the label emoji, numpy array of shape (m, 1)
    
    Returns:
    pred -- numpy array of shape (m, 1) with your predictions
    """
    m = X.shape[0]
    pred = np.zeros((m, 1))
    
    for j in range(m):                       # Loop over training examples
        
        # Split jth test example (sentence) into list of lower case words
        words = X[j].lower().split()
        
        # Average words' vectors
        avg = np.zeros((50,))
        for w in words:
            avg += word_to_vec_map[w]
        avg = avg/len(words)

        # Forward propagation
        Z = np.dot(W, avg) + b
        A = softmax(Z)
        pred[j] = np.argmax(A)
        
    print("Accuracy: "  + str(np.mean((pred[:] == Y.reshape(Y.shape[0],1)[:]))))
    
    return pred

In [ ]:
pip install emoji --upgrade


     |████████████████████████████████| 133kB 22.4MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import emoji
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
X_train, Y_train = read_csv('/content/drive/My Drive/NLP/emojify_data.csv')

In [ ]:
maxLen = len(max(X_train, key=len).split())

In [ ]:
index = 99
print(X_train[index], label_to_emoji(Y_train[index]))

her smile is so charming 😄


In [ ]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('/content/drive/My Drive/NLP/glove.6B.100d.txt')

In [ ]:
import numpy as np
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
np.random.seed(1)

In [ ]:
def sentences_to_indices(X, word_to_index, max_len):   
    m = X.shape[0]                                
    X_indices = np.zeros((m, max_len))    
    for i in range(m):                            
        sentence_words = [w.lower() for w in X[i].split()]
        j = 0
        for w in sentence_words:
            X_indices[i, j] = word_to_index[w]
            j += 1    
    return X_indices

In [ ]:
X1 = np.array(["haha", "lets play cricket", "tonight we are eating pizza"])
X1_indices = sentences_to_indices(X1,word_to_index, max_len = 5)
print("X1 =", X1)
print("X1_indices =", X1_indices)

X1 = ['haha' 'lets play cricket' 'tonight we are eating pizza']
X1_indices = [[171102.      0.      0.      0.      0.]
 [220930. 286375. 111805.      0.      0.]
 [361859. 384374.  58997. 132714. 285806.]]


In [ ]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    
    vocab_len = len(word_to_index) + 1                  
    emb_dim = word_to_vec_map["cucumber"].shape[0]     
    emb_matrix = np.zeros((vocab_len, emb_dim))
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word] 
    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)
    embedding_layer.build((None,))
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [ ]:
def emoji_converter(input_shape, word_to_vec_map, word_to_index):
    sentence_indices = Input(input_shape, dtype='int32')
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    embeddings = embedding_layer(sentence_indices)   
    X = LSTM(128, return_sequences=True)(embeddings)
    X = Dropout(0.4)(X)
    X = LSTM(128, return_sequences=False)(X)
    X = Dropout(0.4)(X)
    X = Dense(7)(X)
    X = Activation('softmax')(X)
    model = Model(inputs=sentence_indices, outputs=X)   
    return model

In [ ]:
model = emoji_converter((maxLen,), word_to_vec_map, word_to_index)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 10)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 10, 100)           40000100  
_________________________________________________________________
lstm (LSTM)                  (None, 10, 128)           117248    
_________________________________________________________________
dropout (Dropout)            (None, 10, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 7)                 903   

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_one_hot = convert_to_one_hot(Y_train, C = 7)

In [ ]:
model.fit(X_train_indices, Y_train_one_hot, epochs = 50, batch_size = 32, shuffle=True)

Epoch 1/50
6/6 [==============================] - 4s 37ms/step - loss: 1.8673 - accuracy: 0.2885
Epoch 2/50
6/6 [==============================] - 0s 31ms/step - loss: 1.6291 - accuracy: 0.2981
Epoch 3/50
6/6 [==============================] - 0s 45ms/step - loss: 1.5317 - accuracy: 0.2306
Epoch 4/50
6/6 [==============================] - 0s 31ms/step - loss: 1.4490 - accuracy: 0.4386
Epoch 5/50
6/6 [==============================] - 0s 26ms/step - loss: 1.2797 - accuracy: 0.4747
Epoch 6/50
6/6 [==============================] - 0s 49ms/step - loss: 1.0338 - accuracy: 0.6260
Epoch 7/50
6/6 [==============================] - 0s 47ms/step - loss: 0.8674 - accuracy: 0.6516
Epoch 8/50
6/6 [==============================] - 0s 45ms/step - loss: 0.6989 - accuracy: 0.7339
Epoch 9/50
6/6 [==============================] - 0s 42ms/step - loss: 0.5939 - accuracy: 0.7512
Epoch 10/50
6/6 [==============================] - 0s 28ms/step - loss: 0.4455 - accuracy: 0.8448
Epoch 11/50
6/6 [============

In [ ]:
x_test = np.array([['I do not like movies'],["I feel lonely"],["Let us go and watch football world cup tonight"],["Honey lets go out for a date"],["She is the most amazing girl"],["Happy birthday Raj"],["This is the best day of my life"],["My mom is the best"]])

In [ ]:
for x in x_test:
    X_test_indices = sentences_to_indices(x, word_to_index, maxLen)
    print(x[0] +' '+  label_to_emoji(np.argmax(model.predict(X_test_indices))))

I do not like movies 😞
I feel lonely 😞
Let us go and watch football world cup tonight ⚾
Honey lets go out for a date 🍴
She is the most amazing girl ❤️
Happy birthday Raj 😄
This is the best day of my life 😄
My mom is the best ❤️


In [ ]:
model.test(x_test, )